In [1]:
# Everything is the same as with_hash, though without the snapshots.
# Only difference is that I use the deterministic ops flag here.

import sys
import numpy as np
import pickle
import os
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, Dropout, GlobalAveragePooling2D,
                                    MaxPooling2D, Activation, Dense, Layer)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
from tensorflow.keras import backend as K 
sys.path.append('../../../python_scripts/')
import correlations

# Set seed value for python and np
seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed_value)
np.random.seed(seed_value)

K.clear_session()
tf.random.set_seed(seed_value)

# Load predict datasets and whitened datasets
x_train_new = np.load('../../../data/cifar10_modified/x_train_new.npy')
y_train_new = np.load('../../../data/cifar10_modified/y_train_new.npy')
x_test_new = np.load('../../../data/cifar10_modified/x_test_new.npy')
y_test_new = np.load('../../../data/cifar10_modified/y_test_new.npy')

# Function to make models
def init_model(architecture: str, seed: int):
    
    if (architecture == 'all_cnn_c'):
        model = Sequential()
        # model.add(Dropout(0.2, input_shape=x_train.shape[1:])) #input shape from keras cifar10 example
        model.add(Conv2D(96, (3, 3), input_shape=x_train_new.shape[1:], padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), 
                         bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), strides=2, padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), strides=2, padding='same',kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(10, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(GlobalAveragePooling2D())
        model.add(Activation('softmax'))
        
        model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, clipnorm=500),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

    else:
        raise

    return model

# Scheduler callback
def scheduler(epoch, lr):
    if (epoch == 200 or epoch == 250 or epoch == 300):
        return lr * 0.1
    return lr

lr_callback = LearningRateScheduler(scheduler)

# Trajectory callback
class Weights_Callback(Callback):
    '''
    Pre: Must define instance_num, layer_arr, x_predict
    '''
    epoch = 0
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch = epoch
    def on_train_batch_begin(self, batch, logs=None):
        if (self.epoch % 50 == 0 and batch % 390 == 0):
            self.model.save_weights('i'+str(instance_num)+'e'+str(self.epoch)+'b'+str(batch)+'.h5')

In [3]:
# All_CNN_C
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
for i in range(10):
    print('Training All_CNN_C with seed', 0)
    K.clear_session()
    tf.random.set_seed(seed_value)
    all_cnn_c = init_model('all_cnn_c', seed=0)

    instance_num = i+1
    layer_arr = [7]

    history = all_cnn_c.fit(
        x_train_new,
            y_train_new,
            batch_size=128,
            epochs=350,
            callbacks=[lr_callback, Weights_Callback()],
            validation_data=(x_test_new, y_test_new),
            shuffle=True)

    all_cnn_c.save('all_cnn_c_instance_'+str(i)+'.h5')
    with open('trainHistoryDict_'+str(i), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

Training All_CNN_C with seed 0
Train on 50000 samples, validate on 10000 samples
Epoch 1/350
50000/50000 [==============================] - 16s 312us/sample - loss: 2.3234 - accuracy: 0.1054 - val_loss: 2.3178 - val_accuracy: 0.1286
Epoch 2/350
18816/50000 [==========>...................] - ETA: 8s - loss: 2.2997 - accuracy: 0.1398

KeyboardInterrupt: 

In [17]:
K.clear_session()
model = init_model('all_cnn_c', seed=0)
model.get_weights()

[array([[[[-0.5250803 , -0.14894307, -0.20490313, ..., -0.43310413,
            0.13803196, -0.1792608 ],
          [-0.21457788, -0.04464722, -0.33446857, ..., -0.09709263,
            0.39158472, -0.08319641],
          [ 0.17623089, -0.04609227,  0.31086355, ...,  0.34121564,
            0.19499035, -0.23728459]],
 
         [[ 0.2350565 ,  0.21941516,  0.02895115, ..., -0.04248137,
           -0.12330475,  0.35500106],
          [ 0.25649795,  0.38822365, -0.16931048, ...,  0.11268101,
            0.15863134, -0.5544683 ],
          [-0.05443111,  0.07794479,  0.3626208 , ...,  0.3007376 ,
           -0.04272068, -0.03800398]],
 
         [[ 0.21909931, -0.08239754, -0.16636926, ...,  0.27146378,
           -0.4536684 , -0.04234046],
          [-0.23520426,  0.013198  , -0.423915  , ..., -0.03440985,
           -0.11631476,  0.01941733],
          [ 0.28470775, -0.1331871 , -0.18268098, ...,  0.025588  ,
            0.05485938,  0.03250592]]],
 
 
        [[[ 0.02083513,  0.3070427

In [18]:
model.load_weights('i0e0b390.h5')
model.get_weights()

[array([[[[-0.5237651 , -0.14834689, -0.20533885, ..., -0.4322236 ,
            0.13764583, -0.17896679],
          [-0.21474671, -0.04481373, -0.33441275, ..., -0.09722685,
            0.3910278 , -0.08432535],
          [ 0.1757788 , -0.04582749,  0.3107221 , ...,  0.3414725 ,
            0.1961461 , -0.23680179]],
 
         [[ 0.23471174,  0.21938622,  0.02906183, ..., -0.04307264,
           -0.12293343,  0.35366246],
          [ 0.25699237,  0.3886942 , -0.16939472, ...,  0.11184879,
            0.15752766, -0.5531011 ],
          [-0.05565065,  0.07639048,  0.362295  , ...,  0.30087757,
           -0.04159814, -0.03780975]],
 
         [[ 0.21757396, -0.08243105, -0.1664906 , ...,  0.27137336,
           -0.45370185, -0.04260191],
          [-0.23409505,  0.01368709, -0.42309827, ..., -0.03402638,
           -0.11633798,  0.01923334],
          [ 0.28583086, -0.13391602, -0.18211919, ...,  0.0255008 ,
            0.05518187,  0.03235025]]],
 
 
        [[[ 0.02180957,  0.3066442

In [19]:
model.load_weights('i1e0b390.h5')
model.get_weights()

[array([[[[-0.5237651 , -0.14834689, -0.20533885, ..., -0.4322236 ,
            0.13764583, -0.17896679],
          [-0.21474671, -0.04481373, -0.33441275, ..., -0.09722685,
            0.3910278 , -0.08432535],
          [ 0.1757788 , -0.04582749,  0.3107221 , ...,  0.3414725 ,
            0.1961461 , -0.23680179]],
 
         [[ 0.23471174,  0.21938622,  0.02906183, ..., -0.04307264,
           -0.12293343,  0.35366246],
          [ 0.25699237,  0.3886942 , -0.16939472, ...,  0.11184879,
            0.15752766, -0.5531011 ],
          [-0.05565065,  0.07639048,  0.362295  , ...,  0.30087757,
           -0.04159814, -0.03780975]],
 
         [[ 0.21757396, -0.08243105, -0.1664906 , ...,  0.27137336,
           -0.45370185, -0.04260191],
          [-0.23409505,  0.01368709, -0.42309827, ..., -0.03402638,
           -0.11633798,  0.01923334],
          [ 0.28583086, -0.13391602, -0.18211919, ...,  0.0255008 ,
            0.05518187,  0.03235025]]],
 
 
        [[[ 0.02180957,  0.3066442

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]